## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,64.67,91,100,1.99,light rain
1,1,Bambous Virieux,MU,-20.3428,57.7575,80.22,69,40,6.91,scattered clouds
2,2,Castro,BR,-24.7911,-50.0119,52.66,94,2,1.41,clear sky
3,3,Torbay,CA,47.6666,-52.7314,35.58,82,20,10.36,few clouds
4,4,Mataura,NZ,-46.1927,168.8643,47.48,97,100,2.55,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bambous Virieux,MU,-20.3428,57.7575,80.22,69,40,6.91,scattered clouds
9,9,Denpasar,ID,-8.6500,115.2167,87.46,74,20,12.66,few clouds
13,13,San Antero,CO,9.3741,-75.7589,75.34,89,100,3.00,overcast clouds
17,17,Oranjestad,AW,12.5240,-70.0270,80.13,84,14,23.33,few clouds
18,18,Bathsheba,BB,13.2167,-59.5167,81.21,78,40,14.97,scattered clouds
20,20,Kapaa,US,22.0752,-159.3190,75.47,85,40,10.36,scattered clouds
21,21,Hailar,CN,49.2000,119.7000,77.22,20,50,4.12,scattered clouds
23,23,Antalaha,MG,-14.9003,50.2788,79.07,66,71,12.17,light rain
24,24,Zhangye,CN,38.9342,100.4517,80.31,13,100,8.61,overcast clouds
25,25,Hilo,US,19.7297,-155.0900,81.28,89,75,3.00,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                210
City                   210
Country                208
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [10]:
blanks = preferred_cities_df[preferred_cities_df["Country"].isna()]
blanks

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
317,317,Henties Bay,NaN,-22.1160,14.2845,75.06,31,0,23.26,clear sky
475,475,Walvis Bay,NaN,-22.9575,14.5053,76.15,23,0,23.02,clear sky


In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(subset=["Country"], inplace=True)
blanks = preferred_cities_df[preferred_cities_df["Country"].isna()]
blanks

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description


In [13]:
preferred_cities_df.count()

City_ID                208
City                   208
Country                208
Lat                    208
Lng                    208
Max Temp               208
Humidity               208
Cloudiness             208
Wind Speed             208
Current Description    208
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bambous Virieux,MU,80.22,scattered clouds,-20.3428,57.7575,
9,Denpasar,ID,87.46,few clouds,-8.6500,115.2167,
13,San Antero,CO,75.34,overcast clouds,9.3741,-75.7589,
17,Oranjestad,AW,80.13,few clouds,12.5240,-70.0270,
18,Bathsheba,BB,81.21,scattered clouds,13.2167,-59.5167,
20,Kapaa,US,75.47,scattered clouds,22.0752,-159.3190,
21,Hailar,CN,77.22,scattered clouds,49.2000,119.7000,
23,Antalaha,MG,79.07,light rain,-14.9003,50.2788,
24,Zhangye,CN,80.31,overcast clouds,38.9342,100.4517,
25,Hilo,US,81.28,broken clouds,19.7297,-155.0900,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name.


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/HotelPy_Database.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))